Segmentation is applied to either 4D or 3D volumes.
- Volumes are read from directories and organized in a <code>np.array</code>;
- A volume subset is analyzed in order to determine the correct <code>threshold</code>;
- The volume is segmented using the newly found <code>threshold</code>;
- Results are imported into the <code>napari</code> viewer.

In [1]:
import numpy as np                  # type: ignore
import napari                       # type: ignore     
import myfunctions as mf            # type: ignore

#### Reading 4D sequence

In [2]:
exp = 'P28A_FT_H_Exp1'
first_slice = 100
last_slice = 104
OS = 'MacOS'

In [3]:
sequence = mf.read_4Dsequence(exp, first_slice=first_slice, last_slice=last_slice, OS=OS)

threshold = mf.find_threshold(sequence, target=5700)

segmented_sequence = mf.segment4D(sequence, threshold, smallest_3Dvolume=10)

mf.save_segmentation_map(segmented_sequence[0,:,:,:], exp, OS=OS)


Finding threshold...
Threshold=2.00 found in 3.14 s


Segmenting and propagating labels...


Backward propagation: 100%|██████████| 54/54 [00:05<00:00, 10.24it/s]



Filtering...
Small agglomerates removed in 0.64 s


Inconsistent agglomerates removal: 100%|██████████| 45/45 [00:05<00:00,  7.50it/s]

In [4]:
viewer = napari.Viewer()

images = [viewer.add_image(sequence, name='Volume', opacity=0.4)]

labels = [viewer.add_labels(segmented_sequence, name='Labels', blending='additive', opacity=0.8)]

settings = napari.settings.get_settings()
settings.application.playback_fps = 5
viewer.dims.current_step = (0, 0)

#### Reading 3D sequence

In [ ]:
exp = 'P28A_FT_H_Exp1'

sequence = mf.read_3Dsequence(exp, time=150)

threshold = mf.find_threshold(sequence)

threshold = 2

segmented_sequence = mf.segment3D(sequence, threshold)

# mf.save_segmentation_map(segmented_sequence, exp, OS='MacOS')

In [3]:
viewer = napari.Viewer()
_ = viewer.add_image(sequence, opacity=0.4)
_ = viewer.add_labels(segmented_sequence, blending='additive', opacity=0.8)
settings = napari.settings.get_settings()
settings.application.playback_fps = 10
viewer.dims.current_step = (0, 0)

#### 4D segmentation algorithm description

4D segmentation is a generalization of the 3D segmentation algorithm. 

<code>segment3D</code> assigns labels to agglomerates in each 2D $(x,y)$ slice separately using the <code>mask</code> function and exploits label propagation and filtering to settle consistent labels between different slices.
- <code>propagate_labels</code> function is used to assign the same labels to overlapping agglomerates in two neighboring slices. For each agglomerate in <code>previous_mask</code>, proceeding in increasing order of agglomerate size, its label is propagated to the agglomerate with largest overlap in <code>previous_mask</code> and to all other agglomerates with overlap > <code>propatation_threshold</code>. The labels of the agglomerates that were not present in <code>previous_mask</code> are redistributed in order to assume the values of the lowest integers possible. The propagation is carried out in two directions (+z and -z) in order to improve consistency.
- Filtering consists in two processes:
    -  <code>remove_isolated_agglomerates</code> cycles through every slice and removes the agglomerates which are present in neither the previous slice nor in the next slice;
    - <code>remove_small_agglomerates</code> removes the agglomerates with volume smaller than <code>smallest_3Dvolume</code>.

<code>segment4D</code> exploits 3D segmentation to segment each volume in the time sequence separately and applies in 4D the steps that are applied in 3D by <code>segment3D</code> with some small differences.
- Each 3D volume is segmented using <code>segment3D</code> and labels are propagated (this time along the time axis, not the z axis) using <code>propagate_labels</code> function.
- Filtering in 4D exploits again the removal of small agglomerates (this time for agglomerates with volume smaller than <code>smallest_4Dvolume</code>) and introduces another filter by means of <code>remove_inconsistent_agglomerates</code>. This function is used to remove the agglomerates that are not appearing consecutively for at least (<code>time_steps</code>) time instants.